In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from datetime import datetime, timedelta
import tensorflow as tf
import random

seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

In [4]:
df = pd.read_csv(r"C:\Users\Bogdan\OneDrive - University of Warwick\Desktop\Projects\(GOV BONDS) Yield Curve Arbitrage\Data\Expected values of predictor variables (6 mo horizon - 31-12-2024)\Work\Personal Saving Rate - NOT STATIONARY\Data.csv")
df['DATE'] = pd.to_datetime(df['DATE'])
df = df.set_index('DATE')
name = "PSAVERT"

data = df[name] # only care about this column
dataset = np.array(data).reshape(-1,1)#convert the column to an array and this will be 1-D array # then you need to rehape to a 2-D array cause minMaxScaler takes 2-D arrays and for that use reshape(-1,1) cause you know you want 1 column and the rows are automaticall calculated to satisfy the 1 columns and beleiv it is just the shape[0]
scaler = MinMaxScaler(feature_range=(0, 1)) #this is called normalizing (an action that is performed on an array). Here you only specify the range.
scaled_data = scaler.fit_transform(dataset) #maps values in the array to a values in the interval (0,1) while keeping the same distribution of dataset. Reason: MLs work best if all data is scaled similarly.
train_data = scaled_data #this is the subset of my scaled data that I will use for taining the ML scaled_data[rows,columns]

In [5]:
x_train = []
y_train = []
window_size = 30 #this means I believe 12 sequential values are good enough for the ML to take in so as to predict the 13th
for i in range(window_size, len(train_data)):
    x_train.append(train_data[i-window_size:i, 0])
    y_train.append(train_data[i, 0])
#print(x_train[:2])
#print(y_train)
#these are lists of arrays

In [6]:
x_train = np.array(x_train) #this is the array with the sequences of values that are used as input for training
y_train = np.array(y_train) #this is the array with the predictions that are used for training
#in a nutshell both arrays are used for training. x is the input and y is the output and the ML learns to mimic the process meaning given 12 values spit the 13th. And because here you don't actually predict anything since all values are known you use this to train the ML meaning the ML takes the data you just inputed as input and output for granted
#the above are singular arrays ok
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1)) #this is an array of arrays #kinda intuitive that you need that because y_train is safe to be a singlular arry cause you match the 12 values to 1 prediction so that 1 prediction can be the first element of an array but the 12 values are the first element of an array too so that is why you need arrays of arrays because for x the "first value" is the 12 values which you gotta stuff into an array cause you need to understand that there are 12 values stuck in there
#print(x_train) this is a 3-D array

In [7]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(window_size, 1))) #50 is the number of units/neurons, return_sequences = True means this layer will return a sequence of vectors each containing the states of the neurons at one timestep of the input sequence that is of length window_size to the next layer  input_shape declares the structure of the input expected. here you have arrays of 12x1
#in short this layer returns a sequence of 12/window_size vectors with each vector containing 50 elements/states representing states of the neurons
#what happens is you feed the first value of the input_sequence(to be determined) to all neurons simultaneously then you save the state of all neurons in a vector (that is the very first vector containing 50 elements/states at timestep 1 which is the first value fed). Aterwards you feed the second value to all neurons, the states might change and save the states to the second vector and so on
model.add(LSTM(50, return_sequences=False))#return_sequences=Flase means the layer will return the vector containing the neuron states at the last timestep
model.add(Dense(25))#25 neurons all connected to the previous layer's neurons,  each neuron in dense does a weighted sum of the inputs it receives, output 25 values computed as a weighted sum of inputs.
model.add(Dense(1))#1 neuron that will output a single value computed as a weighted sum of its inputs.

model.compile(optimizer='adam', loss='mean_squared_error') #this configures the model to aim to minimize the loss function = MSE by adapting the weights assigned to parameters by the Adam algorithm
model.fit(x_train, y_train, batch_size=8, epochs=30) #batch_size = 1 means that I want to take 1 input at a time so one of those 12x1 arrays at a time + model will update its weights after each sample epochs=1 means I want to go through all the x_train dataset once
#also y_train is the results dataset that you calculate the predictions MSE against in the training stage. So the model will take the first array within x_train array, will start off by having random weights (input + recurrent) which will be adjusted to minimize the MSE. This process occurs for the number of batches and what you want to happen is to get to a point where the weights converge to a value
#this ML model skips the validation stage (will be covered as I learn) this means we use the default settings of adam

Epoch 1/30


c:\Users\Bogdan\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.0380
Epoch 2/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0183
Epoch 3/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0131
Epoch 4/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0112
Epoch 5/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0075
Epoch 6/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0076 
Epoch 7/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0045
Epoch 8/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0149
Epoch 9/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0064 
Epoch 10/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0090
Epoch 11/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0040
Epoch 12/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0130
Epoch 13/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0119
Epoch 14/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0037
Epoch 15/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0080
Epoch 16/30
19/1

In [8]:
window_size = 30
input_seq = train_data[-window_size:]
input_seq = np.reshape(input_seq, (1, window_size, 1))

predicted_values = []
for _ in range(6):  # Predict the next 6 months
    pred = model.predict(input_seq)
    predicted_values.append(pred[0,0])  # Assuming the output is [batch, timestep, feature]
    
    # Update input_seq to include the new prediction
    input_seq = np.append(input_seq[:, 1:, :], np.reshape(pred, (1, 1, 1)), axis=1)

# Reverse scaling to get actual values
predicted_values = np.array(predicted_values).reshape(-1, 1)
predicted_values = scaler.inverse_transform(predicted_values)

# Prepare dates for the predictions
last_date = data.index[-1]
date_range = pd.date_range(start=last_date + pd.DateOffset(months=1), periods=6, freq='MS')

# Create a DataFrame for the predicted values
prediction_df = pd.DataFrame(data=predicted_values, index=date_range, columns=['Predictions'])
print(prediction_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
            Predictions
2024-08-01     3.730997
2024-09-01     3.869255
2024-10-01     4.047383
2024-11-01     4.217264
2024-12-01     4.373681
2025-01-01     4.519139
